# **Big Data and AI Project - Home Credit Default Risk Detection - Map Reducers**

## **Extracting datasets from Kaggle**

In [1]:
#Importing Home Credit Default Risk Kaggle datasets
!pip install kaggle
!kaggle
with open("/root/.kaggle/kaggle.json","w") as fp:
  info = '{"username":"shashankmanurao","key":"52eb38fb5caedc18eda01c110d1022db"}'
  fp.write(info)
!kaggle competitions download -c home-credit-default-risk

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
  0% 0.00/524k [00:00<?, ?B/s]
100% 524k/524k [00:00<00:00, 78.2MB/s]
 96% 261M/271M [00:03<00:00, 75.3MB/s]
100% 271M/271M [00:03<00:00, 79.5MB/s]
 92% 70.0M/76.3M [00:01<00:00, 26.8MB/s]
100% 76.3M/76.3M [00:01<00:00, 52.7MB/s]
 96% 104M/109M [00:01<00:00, 59.1MB/s] 
100% 109M/109M [00:01<00:00, 72.7MB/s]
  0% 0.00/36.5k [00:00<?, ?B/s]
100% 36.5k/36.5k [00:00<00:00, 37.7MB/s]
 98% 36.0M/36.8M [00:00<00:00, 22.7MB/s]
100% 36.8M/36.8M [00:00<00:00, 53.9MB/s]
 75% 27.0M/36.1M [00:00<00:00, 34.0MB/s]
100% 3

In [2]:
#Viewing list of files present
! ls

application_test.csv.zip	    installments_payments.csv.zip
application_train.csv.zip	    POS_CASH_balance.csv.zip
bureau_balance.csv.zip		    previous_application.csv.zip
bureau.csv.zip			    sample_data
credit_card_balance.csv.zip	    sample_submission.csv
HomeCredit_columns_description.csv


## **Transforming datasets into .csv format**

In [3]:
#Unzipping necessary datasets
!unzip application_train.csv.zip
!unzip bureau_balance.csv.zip
!unzip bureau.csv.zip
!unzip credit_card_balance.csv.zip
!unzip installments_payments.csv.zip
!unzip POS_CASH_balance.csv.zip
!unzip previous_application.csv.zip

Archive:  application_train.csv.zip
  inflating: application_train.csv   
Archive:  bureau_balance.csv.zip
  inflating: bureau_balance.csv      
Archive:  bureau.csv.zip
  inflating: bureau.csv              
Archive:  credit_card_balance.csv.zip
  inflating: credit_card_balance.csv  
Archive:  installments_payments.csv.zip
  inflating: installments_payments.csv  
Archive:  POS_CASH_balance.csv.zip
  inflating: POS_CASH_balance.csv    
Archive:  previous_application.csv.zip
  inflating: previous_application.csv  


In [4]:
#Mounting drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## **Storing datasets for future use**

In [0]:
#Copying all datasets to drive
!cp application_train.csv /content/drive/My\ Drive/Colab\ Notebooks
!cp bureau_balance.csv /content/drive/My\ Drive/Colab\ Notebooks
!cp bureau.csv /content/drive/My\ Drive/Colab\ Notebooks
!cp credit_card_balance.csv /content/drive/My\ Drive/Colab\ Notebooks
!cp installments_payments.csv /content/drive/My\ Drive/Colab\ Notebooks
!cp POS_CASH_balance.csv /content/drive/My\ Drive/Colab\ Notebooks
!cp previous_application.csv /content/drive/My\ Drive/Colab\ Notebooks

In [6]:
#Importing libraries
import numpy as np
import pandas as pd 
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from statistics import mode
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from timeit import default_timer as timer
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from xgboost import XGBClassifier
from xgboost import plot_importance

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Loading datasets as dataframes

In [7]:
# Read the full datasets as pandas dataframes
app = pd.read_csv('drive/My Drive/Colab Notebooks/application_train.csv')
bureau = pd.read_csv('drive/My Drive/Colab Notebooks/bureau.csv')
bureau_balance = pd.read_csv('drive/My Drive/Colab Notebooks/bureau_balance.csv')
cash = pd.read_csv('drive/My Drive/Colab Notebooks/POS_CASH_balance.csv')
credit = pd.read_csv('drive/My Drive/Colab Notebooks/credit_card_balance.csv')
previous = pd.read_csv('drive/My Drive/Colab Notebooks/previous_application.csv')
installments = pd.read_csv('drive/My Drive/Colab Notebooks/installments_payments.csv')

print('Read all datasets')

Read all datasets


In [0]:
#Converting type of primary ID columns to string
app['SK_ID_CURR']=app['SK_ID_CURR'].astype('str')
bureau[['SK_ID_CURR','SK_ID_BUREAU']]=bureau[['SK_ID_CURR','SK_ID_BUREAU']].astype('str')
bureau_balance['SK_ID_BUREAU']=bureau_balance['SK_ID_BUREAU'].astype('str')
cash[['SK_ID_PREV','SK_ID_CURR']]=cash[['SK_ID_PREV','SK_ID_CURR']].astype('str')
previous[['SK_ID_PREV','SK_ID_CURR']]=previous[['SK_ID_PREV','SK_ID_CURR']].astype('str')
credit[['SK_ID_PREV','SK_ID_CURR']]=credit[['SK_ID_PREV','SK_ID_CURR']].astype('str')
installments[['SK_ID_PREV','SK_ID_CURR']]=installments[['SK_ID_PREV','SK_ID_CURR']].astype('str')

## **Exploratory Data Analysis**

In [0]:
app.head() #view head of app entity

In [0]:
bureau.head() #view head of bureau entity

In [0]:
bureau_balance.head() #view head of bureau_balance entity

In [0]:
cash.head() #view head of cash entity

In [0]:
previous.head() #view head of previous entity

In [0]:
credit.head() #view head of credit entity

In [0]:
installments.head() #view head of installments entity

### **1.Effect of age on repayment**

For this, we would first like to see the frequency distribution of age of all current applicants.

In [0]:
app['CODE_AGE']=(-1*app['DAYS_BIRTH'])/365
# Plotting the histogram
fig = px.histogram(app, x="CODE_AGE", nbins=25)
fig.show()

We would further like to see how the distribution of age between genders.

In [0]:
fig1 = px.histogram(app, x="CODE_AGE", color = "CODE_GENDER", nbins=25)
fig1.show()

Now, let's plot the relationship between age and the target variable. 

In [0]:
plt.figure(figsize = (10, 8))

# KDE plot of loans that were repaid on time
sns.kdeplot(app.loc[app['TARGET'] == 0, 'CODE_AGE'], label = 'target == 0')

# KDE plot of loans which were not repaid on time
sns.kdeplot(app.loc[app['TARGET'] == 1, 'CODE_AGE'] , label = 'target == 1')

# Labeling of plot
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');

In [0]:
# Age information into a separate dataframe
age_data = app.loc[:,['TARGET']]
age_data['CODE_AGE'] = app['CODE_AGE']

# Bin the age data
age_data['AGE_BINNED'] = pd.cut(age_data['CODE_AGE'], bins = np.linspace(20, 70, num = 11))
age_data.head(10)

In [0]:
# Group by the bin and calculate averages
age_groups  = age_data.groupby('AGE_BINNED').mean()
age_groups

In [0]:
plt.figure(figsize = (8, 8))

# Graph the age bins and the average of the target as a bar plot
plt.bar(age_groups.index.astype(str), 100 * age_groups['TARGET'])

# Plot labeling
plt.xticks(rotation = 75); plt.xlabel('Age Group (years)'); plt.ylabel('Failure to Repay (%)')
plt.title('Failure to Repay by Age Group');

#### Insight:
There is a clear trend: younger applicants are more likely to not repay the loan! The rate of failure to repay is above 10% for the youngest three age groups and below 5% for the oldest age group.

### 2**.Housing Type Frequencies**

In [0]:
#Plotting frequencies of types of houses
sns.countplot(app['NAME_HOUSING_TYPE']);
plt.xticks(rotation=45);
plt.title('Counts of housing type')
plt.show()

#### **Insight:** 
Most people are looking to buy a House/apartment

### **3.Contact information**
There are 6 features showing that client provided some contact information, let's see how many ways of contact clients usually provide.

In [0]:
contact_info = app['FLAG_MOBIL'] + app['FLAG_EMP_PHONE'] + app['FLAG_WORK_PHONE'] + app['FLAG_CONT_MOBILE'] + app['FLAG_PHONE'] + app['FLAG_EMAIL']
sns.countplot(contact_info);
plt.title('Count of ways to contact client');

#### Insight: 
Most clients provide 3 ways to contact them and usually minimum is 2, if we don't consider several people who left only 1.

### **4.Amount Credit Plots**

In [0]:
fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=app['AMT_CREDIT']))
fig.update_layout(
    yaxis_title='AMT_CREDIT'
)

The data is severely skewed by outliers. So let's consider only the values within 90 percentile.

In [0]:
fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(
    x=app[app['AMT_CREDIT'] < np.percentile(app['AMT_CREDIT'], 95)]['AMT_CREDIT'],
    marker_color='#3D9970'   
    ))
fig.update_layout(
    yaxis_title='AMT_DATA within 90 percentile'
)

#### **Insight:**
Hovering over the interactive plot tells us the summary statistics for the final credit extended to the client. The median is 491,211 $

In [0]:
plt.figure(figsize=(20,10))
plt.hist(app['AMT_CREDIT']);
plt.title('AMT_CREDIT histogram');

###  **5.Duration of credit**

In [0]:
plt.figure(figsize=(10,6))
plt.title("Distribution of %s" % 'DAYS_CREDIT')
sns.distplot(bureau['DAYS_CREDIT'].dropna(),color='red', kde=True,bins=100)
plt.show() 

#### **Insight:**
Distribution of Duration between Credit Loan Application and Home Credit Application is ranging between 3000 days to 0 days, with an increasing frequency for shorter number of days - a peak at around 300 days (or less than one year).

### **6.Credit Overdue**


In [0]:
plt.figure(figsize=(10,6))
plt.title("Distribution of %s" % 'CREDIT_DAY_OVERDUE')
sns.distplot(bureau['CREDIT_DAY_OVERDUE'].dropna(),color='red', kde=True,bins=100)
plt.show() 


#### Insight:
Most of the credits have 0 or close to 0 days overdue. The maximum number of credit days overdue is ~3000 days.

### **7.Days Employed**

In [0]:
app['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment');

### **8.Days First drawing**

In [0]:
previous['DAYS_FIRST_DRAWING'].plot.hist(title = 'Days First Drawing Histogram');
plt.xlabel('Days First Drawing');

#### **Insight:**
As we can see from the above two plots, there are outlier values which need to be removed

In [0]:
#Removing outliers
app=app.replace({365243: np.nan})
previous=previous.replace({365243: np.nan})

### 9.Education Type


Let's explore what are the educational backgrounds that are more popular among the loan applicants. 


In [0]:
g = sns.catplot(x="NAME_EDUCATION_TYPE", 
                data=app, kind = 'count',
                height=6, aspect=2, palette = "Set2", ci=None)

#### **Insight:**
Most applicants had completed a high level of Special Education like Bacheor's/Master's/PhD

And let's see how education type has an influence on the target variable.

In [0]:
x,y = 'NAME_EDUCATION_TYPE', 'TARGET'

df1 = app.groupby(x)[y].value_counts(normalize=True)
df1 = df1.mul(100)
df1 = df1.rename('percent').reset_index()

g = sns.catplot(x="TARGET", y="percent", col="NAME_EDUCATION_TYPE", col_wrap=3,
                data=df1, kind = 'bar',
                height=2.5, aspect=2, palette = "BuPu", ci=None)

### **10.Income Type**

In [0]:
g = sns.catplot(x="NAME_INCOME_TYPE", 
                data=app, kind = 'count',
                height=6, aspect=2, palette = "Paired", ci=None)

Income Type Vs Target

In [0]:
x,y = 'NAME_INCOME_TYPE', 'TARGET'

df1 = app.groupby(x)[y].value_counts(normalize=True)
df1 = df1.mul(100)
df1 = df1.rename('percent').reset_index()

g = sns.catplot(x="TARGET", y="percent", col="NAME_INCOME_TYPE", col_wrap=3,
                data=df1, kind = 'bar',
                height=2.5, aspect=2, palette = "Accent", ci=None)

#### **Insight:**
The percentage is higher for people in maternity leave and those who are unemployed.  

### **11.Occupation Type**

In [0]:
g = sns.catplot(x="OCCUPATION_TYPE", 
                data=app, kind = 'count',
                height=6, aspect=2, palette = "afmhot", ci=None)
plt.xticks(rotation=45);

#### **Insight:**
Most Applicants were Laborers or Sales staff

And how the target responds to occupation type..

In [0]:
x,y = 'OCCUPATION_TYPE', 'TARGET'

df1 = app.groupby(x)[y].value_counts(normalize=True)
df1 = df1.mul(100)
df1 = df1.rename('percent').reset_index()

g = sns.catplot(x="TARGET", y="percent", col="OCCUPATION_TYPE", col_wrap=4,
                data=df1, kind = 'bar',
                height=2.5, aspect=2, palette = "OrRd", ci=None)

### **12.Number of Family Members**

In [0]:
# Frequency Distribution

g = sns.catplot(x="CNT_FAM_MEMBERS", 
                data=app, kind = 'count',
                height=6, aspect=2, palette = "pastel", ci=None)

In [0]:
ax = sns.countplot(x="CNT_FAM_MEMBERS", hue="TARGET", data=app, palette = "PuBu")

#### **Insight:**
Most applicants had a family size of just 2 members !

### **13.Number of Children**

In [0]:
g = sns.catplot(x="CNT_CHILDREN", 
                data=app, kind = 'count',
                height=6, aspect=2, palette = "pastel", ci=None)

#### **Insight:**
Most people had 0 children. This makes sense given that the most common family size is just 2.

In [0]:
ax = sns.countplot(x="CNT_CHILDREN", hue="TARGET", data=app, palette = "GnBu")

## **User Defined Functions**

In [0]:
#Function to check percentage of missing rows
def missing(df, threshold = 50):
    # Calculate missing stats 
    miss = pd.DataFrame(df.isnull().sum())
    miss['percent'] = 100 * miss[0] / len(df)
        
    # list of missing columns
    missing_columns = (miss[miss['percent']>threshold])
    return missing_columns

In [0]:
#Checking percentage of missing rows in dataset PREVIOUS
missing(previous,threshold=90)

In [0]:
#Dropping high percentage null columns
previous=previous.drop(columns=["RATE_INTEREST_PRIMARY","RATE_INTEREST_PRIVILEGED","DAYS_FIRST_DRAWING"])

In [0]:
#Function to aggregate secondary datasets
def agg2(df, parent_var, df_name):  
    
    # Remove id variables other than grouping variable
    for col in df.columns:
        if col != parent_var and 'SK_ID' in col:
            df = df.drop(columns = col)
    
    # Only want the numeric variables
    numeric_df = df.select_dtypes(include=['float','int','double']).copy()
    if numeric_df.shape[1] != 0:
        numeric_df[parent_var] = df[parent_var]
    # Group by the specified variable and calculate the statistics
        agg_df = numeric_df.groupby(parent_var).agg(['count', 'mean', 'max', 'min', 'sum'])
    
        columns = []

    # Iterate through the variables names
        for var in agg_df.columns.levels[0]:
            if var != parent_var:
            # Iterate through the stat names
                for stat in agg_df.columns.levels[1]:
                # Make a new column name for the variable and stat
                    columns.append('%s_%s_%s' % (df_name, var, stat))
    
        agg_df.columns = columns
    # Select the categorical columns
    catdata = df.select_dtypes(['category','object','bool'])
    if parent_var in catdata.columns:
        catdata=catdata.drop(columns = parent_var)
    
    if catdata.shape[1] != 0:
        
        categorical = pd.get_dummies(catdata)
        
        # Make sure to put the identifying id on the column
        categorical[parent_var] = df[parent_var]
        
        # Groupby the group var and calculate the sum and mean
        categorical = categorical.groupby(parent_var).agg(['sum', 'count', 'mean'])
        
        column_names = []
    
        # Iterate through the columns in level 0
        for var1 in categorical.columns.levels[0]:
            if var1 != parent_var:
                # Iterate through the stats in level 1
                for stat1 in ['sum', 'count', 'mean']:
                    # Make a new column name
                    column_names.append('%s_%s_%s' % (df_name, var1, stat1))
        categorical.columns=column_names
        if numeric_df.shape[1] == 0:
          agg_df1=categorical
        else:
          # Merge on the parent variable
          agg_df1 = agg_df.merge(categorical, on=parent_var, how = 'outer')
    else:
        agg_df1=agg_df   
    
    # Remove the columns with all redundant values
    _, idx = np.unique(agg_df1, axis = 1, return_index=True)
    agg_df1 = agg_df1.iloc[:, idx]
    
    return agg_df1

In [0]:
#Function to call aggregation function and merge aggregated datasets to main dataset
def agg_subset(df, parent_var, df_name, parent_df=None, grandparent_var=None):
    """Aggregate a grandchild dataframe at the grandparent level."""
    if parent_df is None and grandparent_var is None :
        return agg2(df, parent_var, df_name)
    else:
    # set the parent_var as the index of the parent_df for faster merges
        parent_df = parent_df[[parent_var, grandparent_var]].copy().set_index(parent_var)
        
    # Aggregate the numeric variables at the parent level
        df_agg = agg2(df, parent_var, '%s_LOAN' % df_name)
    
    # Merge to get the grandparent variable in the data
        df_agg = df_agg.merge(parent_df, 
                          on = parent_var, how = 'left')
    
    # Aggregate the numeric variables at the grandparent level
        df_info = agg2(df_agg, grandparent_var, '%s_CLIENT' % df_name)
    
  
        gc.enable()
        del df_agg
        gc.collect()
    
    # Drop the columns with all duplicated values
        _, idx = np.unique(df_info, axis = 1, return_index=True)
        df_info = df_info.iloc[:, idx]
    
        return df_info

In [0]:
#Function to calculate True Positive Rate,False Negative Rate,True Negative Rate and False Positive Rate
def metric(cm):
  TPR=cm[1][1]/(cm[1][0]+cm[1][1])
  FNR=cm[1][0]/(cm[1][0]+cm[1][1])
  TNR=cm[0][0]/(cm[0][0]+cm[0][1])
  FPR=cm[0][1]/(cm[0][0]+cm[0][1])
  return [TPR,FNR,TNR,FPR]

## **Custom features - Manual Feature Engineering**

In [0]:
#Customized features
app['LOAN_RATE'] = app['AMT_ANNUITY'] / app['AMT_CREDIT'] 
app['CREDIT_INCOME_RATIO'] = app['AMT_CREDIT'] / app['AMT_INCOME_TOTAL']
app['EMPLOYED_BIRTH_RATIO'] = app['DAYS_EMPLOYED'] / app['DAYS_BIRTH']
app['EXT_SOURCE_SUM'] = app[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].sum(axis = 1)
app['EXT_SOURCE_MEAN'] = app[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis = 1)
app['AMT_REQ_SUM'] = app[[x for x in app.columns if 'AMT_REQ_' in x]].sum(axis = 1)

bureau['LOAN_RATE'] = bureau['AMT_ANNUITY'] / bureau['AMT_CREDIT_SUM']

bureau_balance['PAST_DUE'] = bureau_balance['STATUS'].isin(['1', '2', '3', '4', '5'])
bureau_balance['ON_TIME'] = bureau_balance['STATUS'] == '0'

previous['LOAN_RATE'] = previous['AMT_ANNUITY'] / previous['AMT_CREDIT']
previous["AMT_DIFFERENCE"] = previous['AMT_CREDIT'] - previous['AMT_APPLICATION']

installments['LATE'] = installments['DAYS_ENTRY_PAYMENT'] > installments['DAYS_INSTALMENT']
installments['LOW_PAYMENT'] = installments['AMT_PAYMENT'] < installments['AMT_INSTALMENT']

cash['LATE_PAYMENT'] = cash['SK_DPD'] > 0.0
cash['INSTALLMENTS_PAID'] = cash['CNT_INSTALMENT'] - cash['CNT_INSTALMENT_FUTURE']

credit['OVER_LIMIT'] = credit['AMT_BALANCE'] > credit['AMT_CREDIT_LIMIT_ACTUAL']
credit['BALANCE_CLEARED'] = credit['AMT_BALANCE'] == 0.0
credit['LOW_PAYMENT'] = credit['AMT_PAYMENT_CURRENT'] < credit['AMT_INST_MIN_REGULARITY']
credit['LATE'] = credit['SK_DPD'] > 0.0

## **Converting number of days/months columns to dates**

In [0]:
#Converting days columns of bureau dataframe to timedelta
for col in ['DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE']:
    bureau[col] = pd.to_timedelta(bureau[col], 'D')
    
bureau[['DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE']].head()

In [0]:
# Establish a starting date for all applications at Home Credit
start_date = pd.Timestamp("2020-04-20")
start_date

In [0]:
# Create the date columns for bureau
bureau['bureau_credit_application_date'] = start_date + bureau['DAYS_CREDIT']
bureau['bureau_credit_end_date'] = start_date + bureau['DAYS_CREDIT_ENDDATE']
bureau['bureau_credit_close_date'] = start_date + bureau['DAYS_ENDDATE_FACT']
bureau['bureau_credit_update_date'] = start_date + bureau['DAYS_CREDIT_UPDATE']
# Drop the time offset columns
bureau = bureau.drop(columns = ['DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE'])

In [0]:
# Convert to timedelta
bureau_balance['MONTHS_BALANCE'] = pd.to_timedelta(bureau_balance['MONTHS_BALANCE'], 'D')
# Make a date column
bureau_balance['bureau_balance_date'] = start_date + bureau_balance['MONTHS_BALANCE']
# Drop the time offset columns
bureau_balance = bureau_balance.drop(columns = ['MONTHS_BALANCE'])

In [0]:
# Convert to timedelta
for col in ['DAYS_DECISION', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION']:
    previous[col] = pd.to_timedelta(previous[col], 'D')
    
# Make date columns
previous['previous_decision_date'] = start_date + previous['DAYS_DECISION']
previous['previous_first_due_date'] = start_date + previous['DAYS_FIRST_DUE']
previous['previous_last_duefirst_date'] = start_date + previous['DAYS_LAST_DUE_1ST_VERSION']
previous['previous_last_due_date'] = start_date + previous['DAYS_LAST_DUE']
previous['previous_termination_date'] = start_date + previous['DAYS_TERMINATION']

# Drop the time offset columns
previous = previous.drop(columns = ['DAYS_DECISION', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION'])

In [0]:
# Convert to timedelta objects
credit['MONTHS_BALANCE'] = pd.to_timedelta(credit['MONTHS_BALANCE'], 'D')
cash['MONTHS_BALANCE'] = pd.to_timedelta(cash['MONTHS_BALANCE'], 'D')

# Make a date column
credit['credit_balance_date'] = start_date + credit['MONTHS_BALANCE']
credit = credit.drop(columns = ['MONTHS_BALANCE'])

# Make a date column
cash['cash_balance_date'] = start_date + cash['MONTHS_BALANCE']
cash = cash.drop(columns = ['MONTHS_BALANCE'])

# Convert to time delta object
installments['DAYS_INSTALMENT'] = pd.to_timedelta(installments['DAYS_INSTALMENT'], 'D')
installments['DAYS_ENTRY_PAYMENT'] = pd.to_timedelta(installments['DAYS_ENTRY_PAYMENT'], 'D')

# Create time column and drop
installments['installments_due_date'] = start_date + installments['DAYS_INSTALMENT']
installments = installments.drop(columns = ['DAYS_INSTALMENT'])

installments['installments_paid_date'] = start_date + installments['DAYS_ENTRY_PAYMENT']
installments = installments.drop(columns = ['DAYS_ENTRY_PAYMENT'])

## **Converting categorical columns of APPLICATION dataframe to dummy variables**

In [0]:
#One hot encode application dataframe columns
app = app.set_index('SK_ID_CURR')
cat=app.select_dtypes(['object','category'])
category=pd.get_dummies(cat)
app=app.drop(app.select_dtypes(['object','category']).columns,axis=1)
app=app.merge(category,on="SK_ID_CURR",how='left')
app.head()

## **Aggregating and Merging datasets**

In [0]:
#Aggregating bureau dataframe
bureau_info = agg_subset(df=bureau, parent_var='SK_ID_CURR', df_name='BUREAU')
bureau_info.head()

In [0]:
#Aggregating balance bureau dataframe
bureau_balance_info = agg_subset(bureau_balance, 'SK_ID_BUREAU', "BB", parent_df=bureau, grandparent_var='SK_ID_CURR')
bureau_balance_info.head()

In [0]:
del bureau_balance, bureau

In [0]:
#Merging bureau dataframe to application dataframe
app = app.merge(bureau_info, on = 'SK_ID_CURR', how = 'left')
app.head()

In [0]:
del bureau_info

In [0]:
#Merging bureau balance dataframe to application dataframe
app = app.merge(bureau_balance_info, on = 'SK_ID_CURR', how = 'left')
app.head()

In [0]:
del bureau_balance_info

In [0]:
#Aggregating previous dataframe
previous_info = agg_subset(df=previous, parent_var='SK_ID_CURR', df_name='PREVIOUS')
previous_info.head()

In [0]:
#Merging previous dataframe to application dataframe
app = app.merge(previous_info, on = 'SK_ID_CURR', how = 'left')
app.head()

In [0]:
del previous_info

In [0]:
#Aggregating installments dataframe
installments_info = agg_subset(installments, 'SK_ID_PREV', 'IN', previous,'SK_ID_CURR')
installments_info.head()

In [0]:
del installments

In [0]:
#Merging installments dataframe to application dataframe
app = app.merge(installments_info, on = 'SK_ID_CURR', how = 'left')
app.head()

In [0]:
del installments_info

In [0]:
#Aggregating cash dataframe
cash_info = agg_subset(cash, 'SK_ID_PREV', 'CASH', previous, 'SK_ID_CURR')
cash_info.head()

In [0]:
del cash

In [0]:
#Merging cash dataframe to application dataframe
app = app.merge(cash_info, on = 'SK_ID_CURR', how = 'left')
app.head()

In [0]:
del cash_info

In [0]:
#Aggregating credit dataframe
credit_info = agg_subset(credit, 'SK_ID_PREV', 'CC', previous, 'SK_ID_CURR')
credit_info.head()

In [0]:
del credit, previous

In [0]:
#Merging credit dataframe to application dataframe
app = app.merge(credit_info, on = 'SK_ID_CURR', how = 'left')
app.head()

In [0]:
del credit_info

## **Checking zero variance columns**

In [0]:
corrs=[]
for col in app.columns:
    # Calculate correlation with the target
    cor = app['TARGET'].corr(app[col])
    
    # Append the list as a tuple
    corrs.append((col, cor))
corrs = sorted(corrs, key = lambda x: abs(x[1]))
plt.hist(corrs[:15])
plt.xticks(rotation=90);

In [0]:
#Dropping columns with negligible correlation with the target
app=app.drop(columns=["BUREAU_AMT_CREDIT_SUM_OVERDUE_min","ORGANIZATION_TYPE_Industry: type 7"])

In [0]:
app.shape #check shape of final merged dataframe

## **Checking for Multi-Collinearity**

In [0]:
#Create sample application dataframe to generate correlation matrix
app_sample=app.sample(frac=0.4)

In [0]:
#Generating correlation matrix
corr_matrix = app_sample.corr()

In [0]:
#List of columns with high multi-collinearity
correlation_threshold=0.99
top = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))
high_corr_cols = [col for col in top.columns if any(top[col].abs() > correlation_threshold)]
len(high_corr_cols)

In [0]:
#Dropping highly multi-collinear columns
app=app.drop(columns=high_corr_cols)

In [0]:
#Saving final dataset
app.to_csv("appDF_final3.csv")

In [0]:
!cp appDF_final3.csv drive/My\ Drive/Colab\ Notebooks/

In [0]:
#Loading final dataset
app=pd.read_csv('drive/My Drive/Colab Notebooks/appDF_final3.csv')

In [0]:
app.head() #view head

In [0]:
app.shape     #view updated shape

In [0]:
#Checking for columns with zero values
zero_cols=[i for i in app.columns if app[i].sum()==0]
zero_cols

In [0]:
#Dropping columns with zero values
app=app.drop(columns=zero_cols)

In [0]:
#Removing infinity values
app = app.replace([np.inf, -np.inf], np.nan)

## **Train Test Split**

In [0]:
msk = np.random.rand(len(app)) < 0.8

app_testm = app[~msk]

#Saving TARGET of test
app_testlabel = app_testm["TARGET"]

app1 = app[msk]
#app_trainlabel=app["TARGET"]

## **Undersampling TARGET value 0's due to high bias**

In [0]:
app_ones=app1[app1["TARGET"]==1]


app_zeros=app1[app1["TARGET"]==0]

#Undersampling 0's
app_zeros_sample=app_zeros.sample(frac=0.120907)
#Concatenating TARGET value 0's and 1's rows
app_new=pd.concat([app_ones,app_zeros_sample],join='outer')
#Shuffling rows
app_new=app_new.sample(frac=1)

app_trainlabel=app_new["TARGET"]

In [0]:
app_new=app_new.reset_index()

In [0]:
#Dropping primary ID and dependent variable from train
app_new=app_new.drop(["SK_ID_CURR","TARGET"],axis=1)

In [0]:
app_testm=app_testm.reset_index()

In [0]:
#Dropping primary ID and dependent variable from test
app_testm=app_testm.drop(["SK_ID_CURR","TARGET"],axis=1)

In [0]:
app_testm.shape

## **Imputing train and test**

In [0]:
# Median imputation of missing values
imputer = SimpleImputer(strategy = 'median', missing_values = np.nan)

# Fit on the train data
imputer.fit(app_new)

# Transform on train and test data
app_new = imputer.transform(app_new)
app_testm = imputer.transform(app_testm)


## **Scaling train and test**

In [0]:
# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the train data
scaler.fit(app_new)

# Transform on the train and test data
app_new = scaler.transform(app_new)
app_testm = scaler.transform(app_testm)

# **Modeling**

## **K-Means algorithm Model**

In [0]:
Kmeans=KMeans(n_clusters=50,init="k-means++",max_iter=300,n_init=10,random_state=0) #build model
Kmeans.fit(app_new,app_trainlabel)

In [0]:
predictions_kmeans_train=Kmeans.predict(app_new) #make predictions

In [0]:
XY=pd.DataFrame({"C":predictions_kmeans_train,"Y":app_trainlabel})

In [0]:
def cut_mode(x): #custom function to compute for a cut-off
    if x.mean() > 0.5:
      return 1
    else:
      return 0

XY_frame=XY.groupby("C").agg(cut_mode)

In [0]:
predictions_kmeans=Kmeans.predict(app_testm) #make predictions on test

In [0]:
test_p=[XY_frame.loc[i] for i in predictions_kmeans]
confusion_matrix(app_testlabel,test_p),accuracy_score(app_testlabel,test_p) #compute confusion matrix

In [0]:
precision_recall_fscore_support(app_testlabel,test_p) #compute metrics

## 

## **Logistic Regression Model**

In [0]:
logreg = LogisticRegressionCV(Cs=20, cv=3)
logreg.fit(app_new, app_trainlabel)
predictions_log = logreg.predict_proba(app_testm)[:, 1]

In [0]:
predictions_log = np.where(predictions_log <= 0.5,0,1)

In [0]:
confusion_matrix(app_testlabel,predictions_log),accuracy_score(app_testlabel,predictions_log)

In [0]:
precision_recall_fscore_support(app_testlabel,predictions_log)

## **Principal Component Analysis**

In [0]:
pca = PCA(n_components=3)
reduction = pca.fit_transform(app_new)

fig = pyplot.figure(figsize=(10,10))
ax = Axes3D(fig)
#take = 10000
ax.scatter(reduction[:,0], reduction[:,1], reduction[:,2],c = app_trainlabel,cmap = "coolwarm")

## **Neural Networks Model**

In [0]:
app_new1 = np.eeeexpand_dims(app_new, 2)
app_testm1 = np.expand_dims(app_testm, 2)

In [0]:
app_new1.shape()

In [0]:

#build model with many convolution and dense layers
model = tf.keras.Sequential([
        #Conv1D
        #tf.keras.layers.Convolution1D(512, kernel_size=5,strides=1, activation='relu',batch_input_shape=(None,950,1),kernel_initializer=keras.initializers.glorot_normal()), #give shape
        # BatchNormalization
        #keras.layers.BatchNormalization(),
        #Conv1D
        #tf.keras.layers.Convolution1D(256, kernel_size=5,strides=1, activation='relu',kernel_initializer=keras.initializers.glorot_normal()), #give shape
        # BatchNormalization
        #keras.layers.BatchNormalization(),
        #Conv1D
        tf.keras.layers.Convolution1D(128, kernel_size=5,strides=1, activation='relu',batch_input_shape=(None,950,1), kernel_initializer=keras.initializers.glorot_normal()), #give shape
        # BatchNormalization
        keras.layers.BatchNormalization(),
        #Conv1D
        tf.keras.layers.Convolution1D(64, kernel_size=5,strides=1, activation='relu',kernel_initializer=keras.initializers.glorot_normal()), #give shape
        # BatchNormalization
        keras.layers.BatchNormalization(),
        # 1D Maxpooling
        tf.keras.layers.MaxPooling1D(pool_size=2, strides=1),
        #Conv1D
        tf.keras.layers.Convolution1D(32, kernel_size=5,strides=1, activation='relu',kernel_initializer=keras.initializers.glorot_normal()), #give shape
        # BatchNormalization
        keras.layers.BatchNormalization(),
        #Flatten
        tf.keras.layers.Flatten(),
        #Input layer
        tf.keras.layers.Dense(units=512, activation='relu',kernel_initializer=keras.initializers.glorot_normal()),# input_shape=(948,)), #set shape
        tf.keras.layers.Dropout(0.2), #to avoid overfitting
        #Hidden layer I
        tf.keras.layers.Dense(128, activation='relu',kernel_initializer=keras.initializers.glorot_normal()),
        tf.keras.layers.Dropout(0.2), #to avoid overfitting
        #Ouput Layer
        tf.keras.layers.Dense(1, activation='sigmoid') #activation function to output probabilities '1' or '0'
])

# Compile Model
model.compile(loss = 'binary_crossentropy', optimizer="adam", metrics=['acc','Recall']) #Gradient descent with adaptive learning rate
# Print Model Summary 
model.summary()

In [0]:
#set no. of iterations
eeeeepoch = 8

filepath="weights.hdf5"
checkpointer = ModelCheckpoint(filepath=filepath,monitor = "val_acc", verbose=1, save_best_only=True)
#fit model on train data
history = model.fit(app_new1, app_trainlabel,batch_size=32,epochs=epoch,validation_data=(app_testm1, app_testlabel),callbacks=[checkpointer])

#evaluate metrics on test data
preds_DNN=model.evaluate(app_testm1, app_testlabel)

In [0]:
preds_DNN

In [0]:
preds=model.predict(app_testm1) #make predictions
predictions_nn = np.where(preds <= 0.5,0,1).flatten() 

In [0]:
#compute confusion matrix
confusion_matrix(np.array(app_testlabel).astype(int),predictions_nn.astype(int)),accuracy_score(np.array(app_testlabel).astype(int),predictions_nn.astype(int))

In [0]:
precision_recall_fscore_support(np.array(app_testlabel).astype(int),predictions_nn.astype(int)) #compute metrics

In [0]:
# Plot the chart for accuracy and recall on both training and validation

import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['recall']
val_loss = history.history['val_recall']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Recall')
plt.plot(epochs, val_loss, 'b', label='Validation Recall')
plt.title('Training and validation Recall')
plt.legend()

plt.show()

## **Random Forest Model**

In [0]:
rf = RandomForestClassifier(n_estimators = 700,max_depth=100, n_jobs = -1, verbose = 1) #build model with hyperparameters
rf.fit(app_new, app_trainlabel)

In [0]:
rf_predictions=rf.predict(app_testm) #predict on test data

In [0]:
predictions_rf_test = np.where(rf_predictions <= 0.6,0,1) #set cut-off value

In [0]:
confusion_matrix(app_testlabel,predictions_rf_test),accuracy_score(app_testlabel,predictions_rf_test) #confusion matrix

In [0]:
precision_recall_fscore_support(app_testlabel,predictions_rf_test) #compute metrics

## **XGBoost Model**

In [0]:
model_xg = XGBClassifier(eta=0.1,n_estimators=100,max_depth=10) #build model with hyperparameters
model_xg.fit(app_new, app_trainlabel)

In [0]:
y_predm = model_xg.predict(app_testm) #predict on test set

In [0]:
confusion_matrix(app_testlabel,y_predm),accuracy_score(app_testlabel,y_predm) #compute confusion matrix

In [0]:
precision_recall_fscore_support(app_testlabel,y_predm) #compute metrics

### **XGBoost Feature Importances**

In [0]:
#Extracing feature strengths
feature_imp=pd.DataFrame(model_xg.feature_importances_)
feature_imp

In [0]:
#Extracting features
features=pd.DataFrame(app.columns)
features=features.drop(0)
features=features.reset_index(drop=True)
features

In [0]:
#Merging features and their strengths
features['features']=features[0]

features=features.drop(0,axis=1)

feat=pd.concat([features,feature_imp],axis=1)

feat

In [0]:
#Plotting feature importances and displaying top features in table
plt.plot(feat["features"],feat[0])
plt.xticks(rotation=90);
feat.sort_values(by=0,ascending=False).head(10)

As seen from the above graph and table, EXT_SOURCE features are very important

# **Conclusion**

The XGBoost Model and the Logistic Regression models were the best performing models keeping both accuracy and True positive rate in mind. Our aim was to not have many false negatives i.e. predicting defaulters as non-defaulters which would bring great loss to the firm.
Hence, high True positive rate was desired keeping a decent accuracy figure.
Finally we choose Logistic Regression Model for predictions since it is more simpler and highly interpretable.